<a href="https://colab.research.google.com/github/huishingchong/agile_llm/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Pipeline

## Set up
### Import Packages and API keys

In [18]:
!pip install transformers datasets torch torchvision torchaudio langchain-community faiss-cpu sentence-transformers langchain gradio mlflow evaluate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [1]:
from getpass import getpass
from dotenv import load_dotenv
import os
from pathlib import Path

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

huggingface_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if not huggingface_api_token:
    huggingface_api_token = getpass("Enter your Hugging Face Hub API token: ")

### Load Model

In [2]:
from langchain_community.llms import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint
model_name = "tiiuae/falcon-7b-instruct"
llm = HuggingFaceEndpoint(
    repo_id=model_name,
    model=model_name,
    task="text-generation",
    temperature=0.5,
    # max_length:1024,
    max_new_tokens=200
)
# llm = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature":0.5, "max_length":1024, "max_new_tokens":200})


/Users/huishingchong/agile_llm/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/huishingchong/.cache/huggingface/token
Login successful


## RAG from synthetic data set (skills build)

### Set up documents, embeddings and retrievers

In [3]:
# Use langchain packages to help with implementing retrieval augmentation generation
from datasets import load_dataset
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer

In [4]:
# Load data for retrieval from a csv file
sb_loader = CSVLoader(file_path="skills_build.csv")
sb_documents = sb_loader.load()

# Split documents into chunks
text_split = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
# text_split = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
sb_splitted_documents = text_split.split_documents(sb_documents)

In [5]:
modelPath = "sentence-transformers/gtr-t5-base" # Using t5 sentence transformer model to generate embeddings
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': True} # Normalizing embeddings may help improve similarity metrics by ensuring that embeddings magnitude does not affect the similarity scores

# Initialise an instance of HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [6]:
# Create a FAISS Index with specified document and embeddings
sb_db = FAISS.from_documents(sb_splitted_documents, embedding=embeddings)

In [7]:
# check that retriever works
query_example = "What is IBM SkillsBuild credentials?"
sb_retriever = sb_db.as_retriever()
sb_docs = sb_retriever.get_relevant_documents(query_example)
sb_docs

[Document(page_content="IBM SkillsBuild: Digital credentials\nDescription: Digital credentials from IBM SkillsBuild are secure, web-enabled credentials that contain granular, verified information that employers can use to evaluate an individual's potential. Digital credential requirements can include online learning activities, quizzes or exams, experience that requires advisor review, or even interviews. Complete the required activities, accept the credential, and choose to share your achievements to your resume, LinkedIn, and other social media.", metadata={'source': 'skills_build.csv', 'row': 7}),
 Document(page_content='IBM SkillsBuild: broader initiative\nDescription: IBM SkillsBuild is a free education program focused on underrepresented communities that helps adult learners, high school and university students, and faculty develop valuable new skills and access career opportunities. The program includes an online platform complemented by customised practical learning experiences

In [ ]:
for doc in sb_docs:
    print(f"\nContent: {doc.page_content}, Metadata: {doc.metadata}")

Evaluate retrieval

In [ ]:
sb_docs_and_scores = sb_db.similarity_search_with_score(query_example)
print(sb_docs_and_scores)

In [ ]:
for doc, score in sb_docs_and_scores:
    print(f"\nContent: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

### RetrievalQA

In [7]:
from langchain.prompts import PromptTemplate

# template = """Use the following context to answer the question at the end.
# If you don't know the answer, please think rationally and answer from your own knowledge base.
# Context: {context}

# Question: {question}
# """
template = """Use the following context to answer the question at the end.
If you don't know the answer, please think rationally and answer from your own knowledge base.
Context: {context}

Question: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

In [8]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt": QA_CHAIN_PROMPT}
sb_qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=sb_db.as_retriever(),
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs, verbose=True)


In [ ]:
sb_response = sb_qa.invoke({"query": "What is the responsibility of the Penetration Tester?"})
print(sb_response["result"])

In [ ]:
# See the source documents
print(sb_response.get("source_documents", []))

In [ ]:
import gradio as gr

def chat_interface(textbox, chat):
    input_dict = {'query': textbox}
    result = sb_qa.invoke(input_dict)
    print(result)
    text = result['result']
    return text

gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(debug=True)

## RAG from synthetic data set (job recommendation)

### Set up documents, embeddings and retrievers

In [ ]:
# Load data for retrieval from a csv file
loader = CSVLoader(file_path="sample_jobs.csv")
documents = loader.load()

# Split documents into chunks
text_split = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
d = text_split.split_documents(documents)

In [9]:
modelPath = "sentence-transformers/gtr-t5-base" # Using t5 sentence transformer model to generate embeddings
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': True} # Normalizing embeddings may help improve similarity metrics by ensuring that embeddings magnitude does not affect the similarity scores

# Initialise an instance of HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
# Create a FAISS Index with specified document and embeddings
db = FAISS.from_documents(d, embedding=embeddings)

In [ ]:
# just checking
query_example = "I am a graduate student looking for an internship for software engineering. Please list the skills and qualifications required for a job of such role."
r = db.as_retriever()
docs = r.get_relevant_documents(query_example)


In [ ]:
for doc in docs:
    print(f"\nContent: {doc.page_content}, Metadata: {doc.metadata}")

Evaluate retrieval

In [ ]:
docs_and_scores = db.similarity_search_with_score(query_example)
print(docs_and_scores)

In [ ]:
for doc, score in docs_and_scores:
    print(f"\nContent: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

###RetrievalQA

In [ ]:
from langchain.prompts import PromptTemplate

# template = """Use the following context to answer the question at the end.
# If you don't know the answer, please think rationally and answer from your own knowledge base.
# Context: {context}

# Question: {question}
# """
template = """Use the following context to answer the question at the end.
If you don't know the answer, please think rationally and answer from your own knowledge base.
Context: {context}

Question: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt": QA_CHAIN_PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=db.as_retriever(),
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs, verbose=True)


In [ ]:
response = qa.invoke({"query": query_example})
print(response["result"])

In [ ]:
# See the source documents
print(response.get("source_documents", []))

In [ ]:
import gradio as gr

def chat_interface(textbox, chat):
    input_dict = {'query': textbox}
    result = qa.invoke(input_dict)
    print(result)
    text = result['result']
    return text

gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    examples=["What does AI stand for?", "What is Software Engineering?", "What is Cybersecurity?"],
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(debug=True)

## Evaluation

In [ ]:
!pip install torch tiktoken evaluate mlflow bert_score evaluate textstat


In [ ]:
# RAG evaluation

from langchain.evaluation import load_evaluator, EvaluatorType



Prepare data set

In [10]:
import pandas as pd
data_skills_build = pd.DataFrame({
    "question": [
        "What is the goal of IBM SkillsBuild?",
        "What are the courses available at IBM SkillsBuild?",
        "What is IBM SkillsBuild Software downloads?",
        "What learning paths do IBM SkillsBuild offer to different age groups and how does it support their learning and professional development goals?",
        "How does IBM SkillsBuild support personalized learning experiences for users?",
        "What can learners do with earned digital credentials?",
        "What is a Credly account in IBM SkillsBuild?",
        "What can educators do on IBM SkillsBuild?",
        "What are the specific tools educators can access on IBM SkillsBuild?",
        "What are the specific tools or resources that college students enrolled on IBM SkillsBuild can access to enhance their learning?"
    ],
    "ground_truth": [
        "The goal of IBM SkillsBuild is to address the growing demand for essential technical and professional skills in the face of digital acceleration. It aims to empower learners from underrepresented communities, including adult learners, high school and university students, and faculty, by providing them with access to free education programs. These programs are designed to help individuals develop valuable new skills and access career opportunities. IBM SkillsBuild offers an online platform supplemented by customized practical learning experiences tailored to meet the evolving needs of learners as they progress through their education and career journeys. This initiative is carried out in collaboration with a network of esteemed education partners, including public high schools, non-profit organizations, governments, and corporations, to ensure widespread access and impact.",
        "IBM SkillsBuild offers a comprehensive range of technical and soft skills to prepare individuals for the evolving job market. On the technical front, learners can delve into cutting-edge areas such as AI, Blockchain, Cloud Computing, Cybersecurity, Data Science, Quantum Computing, and Web Development, among others. Meanwhile, the platform also focuses on cultivating essential workplace skills like Agile methodologies, Design Thinking, Professional skills, Job Readiness, and more. With this diverse array of offerings, IBM SkillsBuild equips learners with the expertise and adaptability needed in today's workplace.",
        "IBM SkillsBuild Software downloads is an element within the IBM SkillsBuild program designed to provide learners and educators at degree-granting accredited academic institutions with access to select IBM assets at no-charge for classroom (teaching and learning) and non-commercial research purposes at the academic institution.",
        "Adult Learners is designed for learners (age 18+) seeking employment in the immediate future. It provides adult learners with free job role preparation specifically focused on entry-level tech roles to help them gain meaningful employment, while earning credentials. High School Students is designed for learners at the career exploration stage (aged 13 to 18). It’s designed to help students begin their career exploration with free digital learning on cutting-edge technology and workplace skills. Academia is designed for university students and faculty. Students from IBM’s partner Universities can earn digital credentials, access software, and explore a library of university guest lectures across data science, artificial intelligence, security and more. Learners also have access to expert conversations from IBM volunteers, access to IBM software, and project-based learning (practical exercises to apply learning they have acquired, including labs using IBM technology). Software Downloads is an element within the IBM SkillsBuild program designed to provide learners and educators at degree-granting accredited academic institutions with access to select IBM assets at no-charge for classroom (teaching and learning) and non-commercial research purposes at the academic institution.",
        "The customized practical learning experiences provided by IBM SkillsBuild are designed to empower learners with valuable skills and opportunities for career advancement. Through project-based learning, participants can apply their newfound knowledge in practical exercises and labs using IBM technology. Additionally, insights from IBM Mentors offer guidance on technical expertise, business acumen, and collaboration skills. Learners have access to premium content, including online courses in 22 languages and certifications equivalent to those earned by IBM employees. Moreover, the platform connects individuals with career opportunities such as job fairs, workshops, and seminars, providing support with job interviews, professional portfolio creation, and internship opportunities, ultimately helping learners navigate their educational and career journeys with confidence.",
        "Digital credentials from IBM SkillsBuild can be shared on the learner's resume, LinkedIn, and other social media for peers and employers to see.",
        "Your Credly account is a repository to manage your digital credentials. It's where you claim (accept), store, and transmit your credentials. You can manage which credentials you want to display in your Credly account settings. To participate in the digital credential program, you need to create a Credly account. To accept or manage credentials, create or sign in to your Credly account. In your profile settings, make sure that the email address you use for your IBM SkillsBuild account is registered in your Credly account. To receive emails regarding issued credentials, turn on User Transactional e-mails. If you have an existing Credly account registered under a IBM SkillsBuild email, you can add your other email address to your account and make it the primary one.",
        "Educators can leverage IBM SkillsBuild to access a wealth of free learning resources tailored for their immediate needs. By signing up individually, educators gain access to a diverse array of materials designed to enhance their teaching methodologies and support student learning outcomes. Additionally, educators have the option to register their entire class or organization, unlocking full administrative access and additional features. This includes the ability to monitor students' learning progress, track course completions and monitor hours completed. Whether exploring resources as an individual or registering a class or organization, IBM SkillsBuild provides educators with valuable tools and support to enhance the educational experience and empower learners for success in the digital age.",
        "There is the Teacher toolkit, Career-readiness toolkit, and Curriculumn enhancement.",
        "College students accessing IBM SkillsBuild have a wide range of software tools at their disposal to enhance their learning and skill development. These include IBM Cloud, a comprehensive cloud platform spanning public, private, and hybrid environments, providing hands-on experience with cloud technologies. Additionally, students can utilize tools like ILOG CPLEX for analytical decision support, SPSS Modeller for advanced statistical analysis, and IBM Cognos Analytics for smarter analytics and decision-making. For cybersecurity education, students can access QRadar SIEM for threat detection and response. IBM SkillsBuild also offers educational resources on DB2 database management and various digital credentials to showcase skills acquired through courses. Furthermore, students can benefit from guest lectures by industry experts, further enriching their knowledge and learning experience. Overall, IBM SkillsBuild provides college students with valuable tools, resources, and opportunities to develop essential skills and prepare for future career paths in the digital age."
    ]
})
#TO REVISE
data_cs_industry = pd.DataFrame({
    "question": [
        "What are the latest trends in artificial intelligence and machine learning?",
        "What are some emerging programming languages that are gaining popularity in the industry?",
        "I am a beginner that wants to get into Data Science, where should I start?",
        "I am a final-year Computer Science student wanting to find a graduate role in Cybersecurity. What are the practical skills required for a career in Cybersecurity that are currently in-demand?",
        "What are the essential skills required for a career in cybersecurity?",
        "What are some in-demand technical skills for aspiring data analysts?",
        "What are the career prospects for individuals with expertise in cybersecurity risk management?",
    ],
    "ground_truth": [
        "In the realm of artificial intelligence (AI) and machine learning (ML), several notable trends have emerged recently. Firstly, there's a growing focus on explainable AI (XAI), which aims to make AI models more transparent and understandable to humans, crucial for applications in fields like healthcare and finance where interpretability is paramount. Secondly, federated learning has gained traction, enabling training of ML models across decentralized devices while preserving data privacy, pivotal for IoT and edge computing scenarios. Additionally, reinforcement learning (RL) advancements, particularly in deep RL, have seen remarkable progress, empowering AI systems to make sequential decisions in dynamic environments, with applications spanning robotics, autonomous vehicles, and gaming. Lastly, the integration of AI with other technologies like blockchain for enhanced security and trustworthiness and with quantum computing for tackling complex optimization problems signifies promising directions for future research and innovation in the AI landscape.",
        "Several emerging programming languages are gaining traction in the industry due to their unique features and capabilities. One such language is Rust, known for its emphasis on safety, concurrency, and performance, making it suitable for systems programming where reliability and efficiency are critical. Another language on the rise is Julia, which specializes in numerical and scientific computing, offering high performance comparable to traditional languages like C and Fortran while maintaining a user-friendly syntax and extensive library support. Additionally, Kotlin, a statically typed language interoperable with Java, has become increasingly popular for Android app development, offering modern features and improved developer productivity. Lastly, Swift, developed by Apple, has gained momentum for iOS and macOS development, providing a concise and expressive syntax along with powerful features like optionals and automatic memory management. These emerging languages cater to specific niches and address evolving industry needs, showcasing their growing relevance and adoption in the programming landscape.",
        "Here is a few things to learn about Data Science to get you started: \nLearn Python or R: Choose one as your primary programming language. \nBasic Statistics: Understand mean, median, mode, standard deviation, and probability.\nData Manipulation: Learn Pandas (Python) or dplyr (R) for data cleaning and manipulation.\nData Visualization: Use Matplotlib, Seaborn (Python), or ggplot2 (R) for visualization.\nMachine Learning Basics: Start with linear regression, logistic regression, decision trees, and evaluation metrics.\nPractice: Work on projects using real-world datasets from sources like Kaggle.\nStay Updated: Follow online resources and communities for the latest trends and techniques.",
        "As a final-year Computer Science student aiming for a graduate role in cybersecurity, it's essential to focus on developing practical skills that are currently in high demand in the industry.\nSome of these key skills include:\n\n1. Knowledge of Networking: Understanding networking fundamentals, protocols (such as TCP/IP), and network architecture is crucial for identifying and mitigating security threats. Familiarize yourself with concepts like firewalls, routers, VPNs, and intrusion detection systems (IDS).\n\n2. Proficiency in Operating Systems: Gain proficiency in operating systems such as Linux and Windows, including command-line operations, system administration tasks, and security configurations. Being able to secure and harden operating systems is essential for protecting against common cybersecurity threats.\n\n3. Understanding of Cryptography: Cryptography is at the heart of cybersecurity, so having a solid understanding of encryption algorithms, cryptographic protocols, and cryptographic techniques is vital. Learn about symmetric and asymmetric encryption, digital signatures, hashing algorithms, and their applications in securing data and communications.\n\n4. Penetration Testing and Ethical Hacking: Develop skills in penetration testing and ethical hacking to identify vulnerabilities and assess the security posture of systems and networks. Familiarize yourself with tools and techniques used by ethical hackers, such as Kali Linux, Metasploit, Nmap, and Wireshark.\n\n5. Security Assessment and Risk Management: Learn how to conduct security assessments, risk assessments, and threat modeling to identify, prioritize, and mitigate security risks effectively. Understand risk management frameworks like NIST, ISO 27001, and COBIT, and how to apply them in real-world scenarios.\n\n6. Incident Response and Forensics: Acquire knowledge of incident response procedures, including detection, analysis, containment, eradication, and recovery from security incidents. Understand digital forensics principles and techniques for investigating and analyzing security breaches and cybercrimes.\n\n7. Security Awareness and Communication: Develop strong communication skills to effectively convey cybersecurity concepts, risks, and recommendations to technical and non-technical stakeholders. Being able to raise awareness about cybersecurity best practices and policies is essential for promoting a security-conscious culture within organizations.\n\n8. Continuous Learning and Adaptability: Cybersecurity is a rapidly evolving field, so it's essential to cultivate a mindset of continuous learning and adaptability. Stay updated with the latest threats, trends, technologies, and best practices through professional development, certifications, and participation in cybersecurity communities and events.\n\nBy focusing on developing these practical skills and staying abreast of industry trends and advancements, you'll be well-prepared to pursue a successful career in cybersecurity upon graduation. Additionally, consider obtaining relevant certifications such as CompTIA Security+, CEH (Certified Ethical Hacker), CISSP (Certified Information Systems Security Professional), or others to further enhance your credentials and marketability in the field.",
        "A career in cybersecurity require a broad spectrum of practical skills, including proficiency in network security protocols and tools like firewalls and intrusion detection/prevention systems (IDS/IPS) for safeguarding network infrastructure. Secure coding practices and knowledge of common vulnerabilities are essential for developing secure software applications, with expertise in frameworks like OWASP Top 10 aiding in vulnerability mitigation. Encryption techniques and cryptographic protocols are vital for securing sensitive data, while incident response and digital forensics skills, alongside tools like SIEM systems, enable effective threat detection and response. Proficiency in penetration testing frameworks like Metasploit and security assessment tools is crucial for identifying and remediating security weaknesses, while knowledge of compliance frameworks such as GDPR ensures organizational adherence to cybersecurity regulations. Effective communication and collaboration skills are imperative for conveying cybersecurity risks and recommendations to stakeholders and collaborating with cross-functional teams to implement security measures. Continued learning and staying updated with the latest cybersecurity trends and technologies are key for navigating this ever-evolving field successfully.",
        "In-demand technical skills for data analysts include proficiency in programming languages like Python, R, or SQL for data manipulation, analysis, and visualization. Familiarity with statistical analysis techniques, such as regression analysis, hypothesis testing, and predictive modeling, is essential for deriving insights from data. Knowledge of data querying and database management systems like MySQL, PostgreSQL, or MongoDB is valuable for accessing and organizing large datasets. Expertise in data wrangling techniques, using tools like pandas, dplyr, or data.table, enables cleaning and transforming raw data into actionable insights. Proficiency in data visualization libraries like Matplotlib, ggplot2, or seaborn is crucial for creating informative and visually appealing charts, graphs, and dashboards to communicate findings effectively. Additionally, experience with machine learning frameworks like scikit-learn or TensorFlow, along with knowledge of data mining techniques, enhances the ability to build predictive models and extract patterns from data.",
        "Career opportunities for individuals in cybersecurity risk management include roles such as cybersecurity risk analysts, security consultants, risk managers, compliance officers, and cybersecurity architects. These professionals play a critical role in identifying, evaluating, and prioritizing cybersecurity risks, developing risk mitigation strategies, and ensuring compliance with regulatory requirements and industry standards. With the ever-evolving threat landscape and the increasing complexity of cybersecurity challenges, individuals with expertise in cybersecurity risk management can expect to have a wide range of career opportunities and advancement prospects in both the public and private sectors, including government agencies, financial institutions, healthcare organizations, and consulting firms. Additionally, obtaining relevant certifications such as Certified Information Systems Security Professional (CISSP), Certified Information Security Manager (CISM), or Certified Risk and Information Systems Control (CRISC) can further enhance career prospects and credibility in the field.",
    ]
})

In [11]:
# Helper function: To prevent having to run predictions again, save output to the dataset as a new column 'predictions'
import datasets

def chain_predictions_from_data(llm_chain, eval_data):
  data = eval_data.copy()
  predictions = []  # List to store predictions for all questions
  source_documents = []
  for question in data['question']:
    # Invoke the llm_chain model with the current question
    input_dict = {'query': question}
    response = llm_chain.invoke(input_dict)
    generated_text = response['result']
    predictions.append(generated_text)  # Append the generated text to the predictions list
    print(question)
    print(generated_text)
    documents = response.get("source_documents", [])
    concatenated_page_content = ""
    for document in documents:
    # Append the page_content of each Document to the page_contents array
      concatenated_page_content += document.page_content + "\n"
    source_documents.append(concatenated_page_content)
  data['source_documents'] = source_documents
  data['predictions'] = predictions  # Assign the predictions list to a new column
  return data

In [12]:
rag_data_skills_build = chain_predictions_from_data(sb_qa, data_skills_build)
os.makedirs("skillsbuild", exist_ok=True)
rag_data_skills_build.to_csv("skillsbuild/rag.csv", index=False)



> Entering new RetrievalQA chain...

> Finished chain.
What is the goal of IBM SkillsBuild?
Answer: The goal of IBM SkillsBuild is to help individuals gain the necessary skills to succeed in the modern workforce by providing them with access to free digital learning resources, mentorship, and job opportunities.


> Entering new RetrievalQA chain...

> Finished chain.
What are the courses available at IBM SkillsBuild?
Answer: IBM SkillsBuild provides a wide range of courses, including AI, Blockchain, Cloud Computing, Cybersecurity, Data science, Quantum computing, Web development, and more.


> Entering new RetrievalQA chain...

> Finished chain.
What is IBM SkillsBuild Software downloads?
Answer: IBM SkillsBuild Software downloads is an element within the IBM SkillsBuild program designed to provide learners and educators at degree-granting accredited academic institutions with access to select IBM assets at no-charge for classroom (teaching and learning) and non-commercial research p

In [14]:
rag_data_skills_build

,question,ground_truth,source_documents,predictions
0,What is the goal of IBM SkillsBuild?,The goal of IBM SkillsBuild is to address the ...,IBM SkillsBuild: broader initiative\nDescripti...,Answer: The goal of IBM SkillsBuild is to help...
1,What are the courses available at IBM SkillsBu...,IBM SkillsBuild offers a comprehensive range o...,IBM SkillsBuild: Course catalogue\nDescription...,Answer: IBM SkillsBuild provides a wide range ...
2,What is IBM SkillsBuild Software downloads?,IBM SkillsBuild Software downloads is an eleme...,IBM SkillsBuild: Software downloads\nDescripti...,Answer: IBM SkillsBuild Software downloads is ...
3,What learning paths do IBM SkillsBuild offer t...,Adult Learners is designed for learners (age 1...,IBM SkillsBuild: For High school learners\nDes...,IBM SkillsBuild offers learning paths to diffe...
4,How does IBM SkillsBuild support personalized ...,The customized practical learning experiences ...,IBM SkillsBuild: broader initiative\nDescripti...,Answer: IBM SkillsBuild supports personalized ...
5,What can learners do with earned digital crede...,Digital credentials from IBM SkillsBuild can b...,IBM SkillsBuild: Digital credentials\nDescript...,Answer: Learners can use their digital credent...
6,What is a Credly account in IBM SkillsBuild?,Your Credly account is a repository to manage ...,IBM SkillsBuild: Digital credentials\nDescript...,A Credly account is an account that can be use...
7,What can educators do on IBM SkillsBuild?,Educators can leverage IBM SkillsBuild to acce...,IBM SkillsBuild: broader initiative\nDescripti...,Answer: Educators can access free digital lear...
8,What are the specific tools educators can acce...,"There is the Teacher toolkit, Career-readiness...",IBM SkillsBuild: Software downloads\nDescripti...,Answer: Educators on IBM SkillsBuild can acces...
9,What are the specific tools or resources that ...,College students accessing IBM SkillsBuild hav...,IBM SkillsBuild: broader initiative\nDescripti...,The specific tools and resources that college ...


Generate bert score results table for the data set

In [13]:
from evaluate import load

def generate_bert_results_table(eval_data):
    ref_texts = eval_data['ground_truth']
    predictions = eval_data['predictions']

    # Compute BERTScore
    bertscore = load("bertscore")
    results = bertscore.compute(predictions=predictions, references=ref_texts, model_type="distilbert-base-uncased")

    # Create DataFrame from BERTScore results
    bert_results_table = pd.DataFrame(results, index=range(0, len(results['precision'])))
    bert_results_table['question'] = eval_data['question']

    bert_results_table.drop(columns=['hashcode'], inplace=True)
    bert_results_table = bert_results_table[['question', 'precision', 'recall', 'f1']]

    return bert_results_table

In [14]:
generate_bert_results_table(rag_data_skills_build)

,question,precision,recall,f1
0,What is the goal of IBM SkillsBuild?,0.885881,0.792965,0.836851
1,What are the courses available at IBM SkillsBu...,0.939811,0.815476,0.873240
2,What is IBM SkillsBuild Software downloads?,0.988668,0.997640,0.993133
3,What learning paths do IBM SkillsBuild offer t...,0.837632,0.772414,0.803702
4,How does IBM SkillsBuild support personalized ...,0.852409,0.814007,0.832766
5,What can learners do with earned digital crede...,0.796053,0.810356,0.803141
6,What is a Credly account in IBM SkillsBuild?,0.826150,0.769402,0.796767
7,What can educators do on IBM SkillsBuild?,0.830273,0.790019,0.809646
8,What are the specific tools educators can acce...,0.724205,0.804344,0.762174
9,What are the specific tools or resources that ...,0.803478,0.815081,0.809238


UniEval

In [16]:
!git clone https://github.com/maszhongming/UniEval.git
%cd UniEval
!pip install -r requirements.txt

fatal: destination path 'UniEval' already exists and is not an empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/huishingchong/agile_llm/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/Users/huishingchong/agile_llm/UniEval


In [19]:
# Factual consistency score
from utils import convert_to_json
from metric.evaluator import get_evaluator

task = 'fact'

# src_list = rag_data_skills_build["source_documents"]
# output_list = rag_data_skills_build["predictions"]

# a list of source documents
src_list = ['Peter and Elizabeth took a taxi to attend the night party in the city. \
             While in the party, Elizabeth collapsed and was rushed to the hospital.']
# a list of model outputs (claims) to be evaluataed
output_list = ['Tom was rushed to hospital.']

# Prepare data for pre-trained evaluators
data = convert_to_json(output_list=output_list, src_list=src_list)
# Initialize evaluator for a specific task
evaluator = get_evaluator(task)
# Get factual consistency scores
eval_scores = evaluator.evaluate(data, print_result=True)

AssertionError: Torch not compiled with CUDA enabled

In [22]:
import torch
torch.cuda.is_available()

False

In [ ]:
# to connect to mlflow server, run in terminal: mlflow ui

In [ ]:
import mlflow
from mlflow.models import infer_signature
import pandas as pd

input_columns = [{"question": "string"}]
output_columns = [{"text": "string"}]
signature = infer_signature(input_columns, output_columns)

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("/falcon-evaluation")

# eval_data: dataframe with questions, ground_truth and predictions
def mlflow_evaluation(eval_data):
  with mlflow.start_run() as run:
    mlflow_results = mlflow.evaluate(
        data=eval_data,
        targets="ground_truth",
        predictions="predictions",
        model_type="question-answering",
    )
    # print(f"See aggregated evaluation results below: \n{mlflow_results.metrics}")

    eval_table = mlflow_results.tables["eval_results_table"]
    print(f"See evaluation table below: \n{eval_table}")

In [ ]:
mlflow_evaluation(rag_data_skills_build)

In [ ]:
mlflow_evaluation(rag_data_cs_industry)

Collect + summarise display of evaluation set and predictions into csv?

In [ ]:
rag_data_cs_industry

####Hardware specs

In [ ]:
#disk info
!df -h

In [ ]:
# memory info
!cat /proc/meminfo

In [ ]:
# cpu info
!cat /proc/cpuinfo

In [ ]:
#Graphs to illustrate comparison between models